---
title: "Home Credit Default Risk"
subtitle: "Part 1 | Exploratory Data Analysis | IS 6812"
author: "Adam Bushman (u6049169)"
date: "9/10/2024"
format: 
    html:
        css: styles.css
        theme: simplex
        toc: true
        embed-resources: true
editor:
    render-on-save: true
---


# Introduction

## The Project

#### Business Problem

Home Credit cares deeply about the population without sufficient credit history and aims to improve inclusion for this underrepresented group by designing prediction models for loan repayment, such that capable borrowers are not denied solely due to absence of credit while protecting against defaults.

#### Benefit of a Solution

A more accurate prediction model will help Home Credit a) provide lending terms and b) sufficiently balance risk such that financial inclusion is expanded.

#### Success Metrics

The success of the project will be measured in accurately predicting loan defaults, so that an increase in loan approval rates among this underrepresented group is accompanied by a stable or even reduction in default rates.

#### Analytics Approach

A supervised machine learning classification approach will be used to predict loan default risk, leveraging alternative data sources such as telco and transactional information, with the target variable being loan repayment.

#### Scope

The project will deliver a predictive model for assessing loan repayment abilities.

#### Details

The project will be executed by a team of business analysts, equipped with data processing and modeling skills. The project will feature iterative milestones for data exploration, model development, and evaluation, culminating in a final model by calendar year end.


## My Approach

For this exploratory data analysis, I plan to use the following languages, APIs, and tools:

*   [Quarto](https://quarto.org/): probably "the" notebook of choice for the scientific community; built and maintained by Posit, it's flexible, powerful, and beautiful.
*   [Python](https://www.python.org/): I usually default to R for analytical tasks so I could use the extra practice in Python.
*   [DuckDB](https://duckdb.org/): an in-process analytical database engine. It is extremely fast and features a convenient syntax. It's gathered tremendous steam in the industry, even for analysis tasks.

::: {layout="[[3,4,4,5]]" layout-valign="center"}
![](https://quarto-dev.github.io/quarto-r/logo.png)

![](https://quantumzeitgeist.com/wp-content/uploads/pythoned.png)

![](https://duckdb.org/images/logo-dl/DuckDB_Logo.png)
::::

Within the Python ecosystem I'll use several libraries that will augment the analytical process. These include, but are not limited to:

*   [pandas](https://pandas.pydata.org/docs/index.html): probably *the* foundational package for data analysis in Python
*   [scikit-learn](https://scikit-learn.org/stable/): a comprehensive API for data science models and workflows
*   [statsmodels](https://www.statsmodels.org/stable/index.html): one of the best statistics libraries in Python
*   [plotnine](https://plotnine.org/): the {ggplot2} equivalent in Python, bringing "grammar of graphics"
*   [skimpy](https://pypi.org/project/skimpy/): a neat package that started in R designed to summarize tabular data in a digestible way


::: {layout="[[8,14,5,5]]" layout-valign="center"}
![](https://numfocus.org/wp-content/uploads/2016/07/pandas-logo-300.png)

![](https://miro.medium.com/v2/resize:fit:984/1*OTt5dpD5N4Ru1qn0WNsZ4g.png)

![](https://plotnine.org/images/logo-540.png)

![](https://docs.ropensci.org/skimr/logo.png)
::::

# Analysis Preparation

## Data Source

The data is sourced from [Kaggle](https://www.kaggle.com/competitions/home-credit-default-risk/overview) where [Home Credit](https://www.homecredit.net/) hosted a competitiion back in the summer of 2018. Data was downloaded therefrom on August 20th, 2024 and used concurrently throughout the Capstone course at University of Utah, Fall 2024.


## Loading Data

We'll start off importing `duckdb`. We'll do much of the data processing work using this powerful SQL engine.


In [ ]:
import duckdb

With access to the API, we can begin to query our data located in files. We need only setup "relations" (informing `duckdb` where the files are located).


In [ ]:
#| output: false

# DuckDB relations
# These are tables against which we can query using the DuckDB API

duckdb.read_csv("data/application_test.csv")
duckdb.read_csv("data/application_train.csv")
duckdb.read_csv("data/bureau_balance.csv")
duckdb.read_csv("data/bureau.csv")
duckdb.read_csv("data/credit_card_balance.csv")
duckdb.read_csv("data/installments_payments.csv")
duckdb.read_csv("data/POS_CASH_balance.csv")
duckdb.read_csv("data/previous_application.csv")

duckdb.read_csv("data/HomeCredit_columns_description.csv")

This approach isn't what you'd call "conventional". How are we expecting to work with this data? We'll walk through some use cases quickly to demonstrate how seamless it will actually be.


## Working With the Data

Straight away, we can interact with these files with plain SQL. For example, with a simple query (below) we can look at the first 10 rows of the `bureau.csv` file.


In [ ]:
duckdb.sql("SELECT * FROM 'data/bureau.csv' LIMIT 10").show()

DuckDB does a nice job styling the output and including helpful details, such as data types.

But don't think we're "stuck" in the world of SQL for the duration of this analysis; far from it. DuckDB is very adept at refactoring the data for use with the other packages we'll be using. Let's bring in `pandas` and see how this works:


In [ ]:
import pandas as pd

In [ ]:
df = duckdb.sql("SELECT * FROM 'data/bureau.csv' LIMIT 10").fetchdf()

type(df)

As you can see, we now have a `pandas` data frame. We could run a model with `scikit-learn`, generate a visualization with `plotnine`, or perform complex, custom logic ill-fit for SQL. For now, we'll just select every other column:


In [ ]:
df_sub = df.loc[:,::2]

df_sub.info()

At this point, we may be ready to leverage the speed and efficiency of DuckDB. So we can just switch right back!


In [ ]:
duckdb.sql("SELECT * FROM df_sub").show()

There really won't be any issue navigating between the different APIs. In fact, we'll be able to leverage the unique strengths of each of them to best advance our analysis. 


# Exploration

## Skim each dataset

We'll start off by getting familiar with each of the data sets. We'll use the `{skimpy}` package for this and do some visualizations.

### `application_train.csv`

This file is the same as `application_test.csv` except that it features the target variable. We'll start with the columns:


In [ ]:
duckdb.sql("""
    SELECT *
    FROM 'data/HomeCredit_columns_description.csv'
""")

In [ ]:
from skimpy import skim

app_train_df = (
    duckdb                                                 # Use DuckDB's query engine
    .sql("SELECT * FROM 'data/application_train.csv'")     # Read the file
    .fetchdf()                                             # Convert to a pandas dataframe
)

skim(app_train_df)                                         # "Skim" the data set

This is a LARGE data set. Let's break down some of the observations we can glean from the summary:

#### Data Types

*   There's several variables casted as numeric that should be categorical
    *   Any variables with `FLAG_` prefixes: ~32 columns
        *   Examples: `FLAG_MOBIL`, `FLAG_DOCUMENT_#`, ...
    *   Other variables treated as flags but not specified in the name: ~6 columns
        *   Examples: `REG_REGION_NOT_LIVE_REGION`, `LIVE_CITY_NOT_WORK_CITY`, ...
    *   `REGION_RATING_` variables that are ordinal classificaitons
*   Many variables of type `string` could be candidates for categorical depending on count of unique values
    *   Examples: `NAME_CONTRACT_TYPE`, `ORGANIZATION_TYPE`
*   Some variables casted as `string` but may need to be numeric
    *   Any with `_MODE` suffix

#### Missing Data

*   `EXT_SOURCE_#` these are scores ingested from third parties
    *   Understandably sparse; not every client would be represented in third-party systems
*   Fields including `APARTMENT_`, `BASEMENT_`, `ELEVATOR_`, etc., all relate to the client's current dwelling.
    *   43 of the file's 122 columns relate to the client's dwelling
    *   These are very sparse, about ~50% complete. These fields represent most of the missing data
    *   Presumably, the lender was unable to collect such detail for every client
*   `OBS_##_SOCIAL_CIRCLE` and `DEF_##_SOCIAL_CIRCLE` relate to default rates on observation ## days past due
    *   This is a count that's heavily skewed to the right
    *   <1% of observations have these fields as missing
*   `AMT_REQ_CREDIT_BUREAU_XX` are fields measure the number of credit inqueries in XX timeframe
    *   This data set is geared around users without sufficient credit history so makes sense this is missing
    *   ~13% of the data is missing
*   `OCCUPATION_TYPE` refers to the type of job the user has
    *   Missing for virtually 1/3 clients
    *   `ORGANIZATION_TYPE`, however, is not missing so we do retain some attribute relative to their work
    *   Perhaps in combination with `NAME_EDUCATION_TYPE` there's predictive power

#### Distribution

*   All dwelling related variables have been *normalized* based on description column descriptions file
    *   Checking the histograms to the far right, this appears to be accurate
*   There's a couple variables with skewed distributions
    *   `DAYS_REGISTRATION`: skewed right
    *   `DAYS_LAST_PHONE_CHANGE`: skewed right

#### Potential feature engineering

*   Dimensionality reduction
    *   `FLAG_DOCUMENT_#` fields could potentially be summarized a few ways;
        *   % of documents completed
        *   Finding principal components
        *   Maybe only certain documents matter
    *   There's many measures of central tendency for dwelling (avg, mode, median)
        *   Likely not all for each perspective is needed

### `bureau.csv`

This file is the same as `application_test.csv` except that it features the target variable.


In [ ]:
from skimpy import skim

app_train_df = (
    duckdb                                                 # Use DuckDB's query engine
    .sql("SELECT * FROM 'data/application_train.csv'")     # Read the file
    .fetchdf()                                             # Convert to a pandas dataframe
)

skim(app_train_df)                                         # "Skim" the data set